# Setup

Regural Expression module

In [1]:
import re

Pseudo random generator module

In [2]:
import random

State machine module

In [3]:
from pysm import State, StateMachine, Event

Natural language module to generate languages with context free grammar

In [4]:
from nltk.parse.generate import generate
from nltk import CFG

numerical computation module

In [5]:
import numpy as np

# Machines, grammar and language

In [6]:
def machine_frame(sm, bit_str):
    sm.initialize()
    for c in bit_str:
        if c == "0":
            sm.dispatch(Event("0"))
        elif c == "1":
            sm.dispatch(Event("1"))
        else:
            raise Exception("Invalid input")
    return sm.state.name

## Finite State Machines (Example)

### For a language of all bit strings with even numbered zeros

Create a 
- Regular Grammar and its complement
- Finite state machine
- Regular exression

### Grammar

In [7]:
grammar_str = """
A -> '1' A | '0' B | 
B -> '1' B | '0' A
"""
grammar = CFG.fromstring(grammar_str)

complement_grammar_str = """
A -> '1' A | '0' B
B -> '1' B | '0' A | 
"""
complement_grammar = CFG.fromstring(complement_grammar_str)

In [8]:
language = []
for i, sentence in enumerate(generate(grammar, depth = 5)):
    language.append("".join(sentence))
language

['1111',
 '111',
 '1100',
 '11',
 '1010',
 '1001',
 '100',
 '1',
 '0110',
 '0101',
 '010',
 '0011',
 '001',
 '0000',
 '00',
 '']

In [9]:
complement_language = []
for i, sentence in enumerate(generate(complement_grammar, depth = 5)):
    complement_language.append("".join(sentence))
complement_language

['1110',
 '1101',
 '110',
 '1011',
 '101',
 '1000',
 '10',
 '0111',
 '011',
 '0100',
 '01',
 '0010',
 '0001',
 '000',
 '0']

#### Regexp

In [10]:
regexp = "^(1*01*01*|1*)*$"
test_result = map(lambda x: True if re.match(regexp, x) is not None else False, language)
for r in test_result:
    print(r)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [11]:
test_result = map(lambda x: True if re.match(regexp, x) is not None else False, complement_language)
for r in test_result:
    print(r)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


### Machine

In [12]:
s_even = State("even")
s_odd = State("odd")

In [13]:
sm = StateMachine("sm")

In [14]:
sm.add_state(s_even, initial=True)
sm.add_state(s_odd)

In [15]:
sm.add_transition(s_even, s_odd, events=["0"])
sm.add_transition(s_odd, s_even, events=["0"])

In [16]:
machine_frame(sm, "010")

'even'

In [17]:
machine_frame(sm, "01001")

'odd'

In [18]:
machine_result = map(lambda x: machine_frame(sm, x), language)
for r in machine_result:
    print(r)

even
even
even
even
even
even
even
even
even
even
even
even
even
even
even
even


In [19]:
machine_result = map(lambda x: machine_frame(sm, x), complement_language)
for r in machine_result:
    print(r)

odd
odd
odd
odd
odd
odd
odd
odd
odd
odd
odd
odd
odd
odd
odd


## For a language of all bit strings that start with `0` and ends with `1` (exercise)

Create a 
- Regular Grammar and its complement
- Finite state machine
- Regular exression

Do which you feel the easiest first

## Push-Down machinmes

## For a language of equal number of `0`s and `1` (Example)

Create a 
- Context Free Grammar
- Deterministic Push-down machine

Do which you feel the easiest first

In [20]:
grammar_str_3 = """
S -> '0' A | '1' B | 
A -> S A | A S | '1'
B -> S B | B S | '0'
"""
grammar_3 = CFG.fromstring(grammar_str_3)

In [21]:
language_3 = []
for i, sentence in enumerate(generate(grammar_3, depth = 7)):
    language_3.append("".join(sentence))
language_arr_3 = np.unique(np.array(language_3))
language_arr_3

array(['', '0001101011', '00011011', '0001101101', '0001101110', '000111',
       '0001110011', '00011101', '0001110101', '0001110110', '00011110',
       '0001111001', '0001111010', '0010101011', '00101011', '0010101101',
       '0010101110', '001011', '0010110011', '00101101', '0010110101',
       '0010110110', '00101110', '0010111001', '0010111010', '0010111100',
       '0011', '0011001011', '00110011', '0011001101', '0011001110',
       '001101', '0011010011', '00110101', '0011010101', '0011010110',
       '00110110', '0011011001', '0011011010', '0011011100', '001110',
       '0011100011', '00111001', '0011100101', '0011100110', '00111010',
       '0011101001', '0011101010', '0011101100', '00111100', '01',
       '0100101011', '01001011', '0100101101', '0100101110', '010011',
       '0100110011', '01001101', '0100110101', '0100110110', '01001110',
       '0100111001', '0100111010', '0100111100', '0101', '0101001011',
       '01010011', '0101001101', '0101001110', '010101', '0101010

### Machine

In [22]:
pdm_1 = StateMachine("pdm_1")

In [23]:
s_push = State("push")
s_pop = State("pop")
s_true = State("true")

In [24]:
pdm_1.add_state(s_push, initial=True)
pdm_1.add_state(s_pop)
pdm_1.add_state(s_true)

Transition condition functions to make our machine see the stack

In [25]:
def stack_is_empty(s, e):
    try:
        pdm_1.stack.peek()
        return False
    except IndexError:
        return True
    
def stack_peek_a(s, e):
    try:
        if pdm_1.stack.peek() == "a":
            return True
        else:
            return False
    except IndexError:
        return False
    
def stack_peek_b(s, e):
    try:
        if pdm_1.stack.peek() == "b":
            return True
        else:
            return False
    except IndexError:
        return False

In [26]:
# push state
## stay
pdm_1.add_transition(
    s_push, s_push, 
    events=["0"],
    condition= lambda s, e : stack_is_empty(s, e) or stack_peek_a(s, e),
    action= lambda s, e : pdm_1.stack.push("a"))

pdm_1.add_transition(
    s_push, s_push, 
    events=["1"],
    condition= lambda s, e : stack_is_empty(s, e) or stack_peek_b(s, e),
    action= lambda s, e : pdm_1.stack.push("b"))
# move to push
pdm_1.add_transition(
    s_push, s_pop, 
    events=["0"],
    condition= lambda s, e : stack_peek_b(s, e),
    action= lambda s, e : pdm_1.stack.pop())

pdm_1.add_transition(
    s_push, s_pop, 
    events=["1"],
    condition= lambda s, e : stack_peek_a(s, e),
    action= lambda s, e : pdm_1.stack.pop())

# pop state
## stay
pdm_1.add_transition(
    s_pop, s_pop, 
    events=["0"],
    condition= lambda s, e : stack_peek_b(s, e),
    action= lambda s, e : pdm_1.stack.pop())

pdm_1.add_transition(
    s_pop, s_pop, 
    events=["1"],
    condition= lambda s, e : stack_peek_a(s, e),
    action= lambda s, e : pdm_1.stack.pop())
## move to push
pdm_1.add_transition(
    s_pop, s_push, 
    events=["0"],
    condition= lambda s, e : stack_is_empty(s, e) or stack_peek_a(s, e),
    action= lambda s, e : pdm_1.stack.push("a"))

pdm_1.add_transition(
    s_pop, s_push, 
    events=["1"],
    condition= lambda s, e : stack_is_empty(s, e) or stack_peek_b(s, e),
    action= lambda s, e : pdm_1.stack.push("b"))
## move to true
pdm_1.add_transition(
    s_pop, s_true,
    events=["e"],
    condition= lambda s, e : stack_is_empty(s, e))

In [27]:
def pmd_1_frame(sm, bit_str):
    sm.initialize()
   
    # clear stack
    while True:
        try:
            sm.stack.pop()
        except IndexError:
            break
    
    for c in bit_str:
        if c == "0":
            sm.dispatch(Event("0"))
        elif c == "1":
            sm.dispatch(Event("1"))
        elif c == "e":
            sm.dispatch(Event("e"))
        else:
            raise Exception("Invalid input")
    try:
        sm.stack.peek()
        return False
    except IndexError:
        return True

In [28]:
pmd_1_frame(pdm_1, '011000011101e')

True

## For the language (Exercise):

$0^{n}10^{n}$

Create a 
- Context Free Grammar
- Deterministic Push-down machine

Do which you feel the easiest first

# Constituency Grammar

## Declarative sentence (example)

In [29]:
def show_random_sentence(sentences):
    rand_ind = random.randint(0, len(sentences))
    return sentences[rand_ind]

In [30]:
grammar_str = """
S -> NP VP PP
Wh -> 'where'
NP -> ProperNoun
NP -> Det N
NP -> Adj ProperNoun
NP -> Det Adj N
PP -> Adp NP
VP -> 'slept'
VP -> V NP
VP -> 'walked' PP
Det -> 'the'
Det -> 'a'
V -> 'rode'
V -> 'discovered'
V -> 'Walked'
v -> 'Seduced'
ProperNoun -> 'Trump'
ProperNoun -> 'Kerrigan'
ProperNoun -> 'Joan of Arc'
ProperNoun -> 'Jack Ketch'
N -> 'chicken'
Adj -> 'pretty'
Adj -> 'magnificent'
Adj -> 'enraged'
Adj -> 'suprised'
Adp -> 'in'
Adp -> 'with'
Adp -> 'in'
Adp -> 'by'
"""

grammar_cfg = CFG.fromstring(grammar_str)

In [31]:
language_n = []
for sentence in generate(grammar_cfg):
    language_n.append(' '.join(sentence))

In [32]:
show_random_sentence(language_n)

'the enraged chicken walked in enraged Jack Ketch in Joan of Arc'

## Question (exercise)

Define a Context Free Grammar grammar which can generate syntacticly valid English questions